# acquire & summarize

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from acquire import get_connection, get_zillow_data

import warnings
warnings.filterwarnings("ignore")

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

    - Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    - Only include properties that include a latitude and longitude value.

In [2]:
df = get_zillow_data()

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [ ]:
df.head()

dropping duplicate id column

In [3]:
df = df.drop(df.columns[[59]], axis = 1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77380 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77380 non-null  float64
 4   heatingorsystemtypeid         49439 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77380 non-null  float64
 10  bedroomcnt                    77380 non-null  float64
 11  buildingqualitytypeid         49671 non-null  float64
 12  calculatedbathnbr             76771 non-null  float64
 13  d

In [ ]:
df.describe().T

In [ ]:
for col in df.columns:
    if df[col].dtype != object:
        plt.figure(figsize=(4,3))
        plt.hist(df[col])
        plt.title(col)
        plt.show()

In [ ]:
for col in df.columns:
    print("\n")
    print(df[col].value_counts())
    print("\n")

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [5]:
def null_finder_columns(df):
    '''
    This function takes in a DataFrame and list 
    information about the null values in the columns
    '''
    #accepts a 'df' and creates a new one labeled 'nulls'  
    #nulls index is the df's columns
    nulls = pd.DataFrame(index = df.columns)
    #sums up the null values in the dataframes columns
    nulls['num_rows_missing'] = df.isnull().sum(axis = 0)
    #finds the percentage of null values in the df's columns
    nulls['pct_rows_missing'] = nulls.num_rows_missing / df.shape[0]
    return nulls

In [11]:
columns = null_finder_columns(df)

In [12]:
columns

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77158,0.997131
storytypeid,77330,0.999354
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,0.361088
...,...,...
buildingclassdesc,77365,0.999806
heatingorsystemdesc,27941,0.361088
propertylandusedesc,0,0.000000
storydesc,77330,0.999354


Takeaways: 
- There are 34 out of the 68 columns missing more than half the value
- May need to drop a lot of the data

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
def nulls_finder_rows(df):
    rows = pd.DataFrame()

    rows['num_cols_missing'] = df.isnull().sum(axis=1)

    rows['pct_cols_missing'] = df.isnull().sum(axis=1) / df.shape[1]

    num_rows = rows.groupby('num_cols_missing').count()

    num_rows = num_rows.rename(columns ={'pct_cols_missing': "num_rows"})

    pct_cols = rows.groupby('num_cols_missing').mean()

    result = pd.concat([pct_cols, num_rows], axis=1, sort=False)

    result = result.reset_index()
    return result

In [9]:
rows = nulls_finder_rows(df)

In [10]:
rows

,num_cols_missing,pct_cols_missing,num_rows
0,23,0.338235,2
1,24,0.352941,13
2,25,0.367647,24
3,26,0.382353,65
4,27,0.397059,316
5,28,0.411765,455
6,29,0.426471,5270
7,30,0.441176,3455
8,31,0.455882,9891
9,32,0.470588,12579


Takeaways:
- There are on average 32 columns with missing values in each rows

# Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
df.propertylandusedesc.value_counts()

In [ ]:
df = df[df.propertylandusedesc == "Single Family Residential"]

In [ ]:
df.bedroomcnt.value_counts()

In [ ]:
df = df[df.bedroomcnt < 8]

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

    - The input:
        - A dataframe
        - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
        - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
    - The output:
        - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
    - hint:
        - Look up the dropna documentation.
        - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
        - Make use of inplace, i.e. inplace=True/False.

In [24]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    thresh_col = int(round(prop_required_column*df.shape[0],0))
    df.dropna(axis=1, thresh=thresh_col, inplace=True)
    thresh_row = threshold = int(round(prop_required_row*df.shape[1],0))
    df.dropna(axis=0, thresh=thresh_row, inplace=True)
    return df 

3. Decide how to handle the remaining missing values:

    - Fill with constant value.
    - Impute with mean, median, mode.
    - Drop row/column